In [1]:
from datetime import datetime
from functools import partial
from pathlib import Path

import argparse
from loguru import logger as log
from mmengine import Config
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from logic.datamodules import EMSDataModule
from logic.io import read_raster_profile, write_raster

from logic.modules.single import SingleTaskModule
from logic.tiling.tilers import SmoothTiler, SimpleTiler
from logic.utils import exp_name_timestamp, find_best_checkpoint

import pathlib

c:\Users\EDELLAA6Y\multitask\.venv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\EDELLAA6Y\multitask\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os 
os.chdir("C:\\Users\\EDELLAA6Y\\multitask\\refactoring2\\burned")

In [10]:
cli = argparse.ArgumentParser()
cli.add_argument("mode", choices=["train", "test"])
cli.add_argument("-c", "--config_path", type=Path)
cli.add_argument("-e", "--experiment_path", type=Path)
cli.add_argument("-cp", "--checkpoint_path", type=Path)
cli.add_argument("-p", "--predict", action="store_true")

_StoreTrueAction(option_strings=['-p', '--predict'], dest='predict', nargs=0, const=True, default=False, type=None, choices=None, required=False, help=None, metavar=None)

In [19]:
train_str = "train -c configs\multi\pretrained\dice\ems_segformer-mit-b3_multi_50ep.py"
args = cli.parse_args(train_str.split())
cfg_path = args.config_path

In [20]:
log.info(f"Loading config from: {cfg_path}")
config = Config.fromfile(cfg_path)

2024-04-28 17:22:59.573 | INFO     | __main__:<module>:1 - Loading config from: configs\multi\pretrained\dice\ems_segformer-mit-b3_multi_50ep.py


In [21]:
exp_name = exp_name_timestamp(config["name"])
config["name"] = exp_name
log.info(f"Experiment name: {exp_name}")

2024-04-28 17:23:12.712 | INFO     | __main__:<module>:3 - Experiment name: segformer-mit-b3_multi_imnet_auxv2_50ep_20240428_172312


In [22]:
# datamodule
log.info("Preparing the data module...")
datamodule = EMSDataModule(**config["data"])

2024-04-28 17:23:14.624 | INFO     | __main__:<module>:2 - Preparing the data module...


In [23]:
datamodule

In [16]:
# prepare the model
log.info("Preparing the model...")
model_config = config["model"]
loss = config["loss"] if "loss" in config else "bce"
model_config

2024-04-27 16:36:26.472 | INFO     | __main__:<module>:2 - Preparing the model...


{'type': 'CustomEncoderDecoder',
 'data_preprocessor': None,
 'backbone': {'type': 'ResNet',
  'pretrained': 'pretrained/mmseg_rn50_ss4eo.pth',
  'depth': 50,
  'in_channels': 12,
  'num_stages': 4,
  'out_indices': (0, 1, 2, 3),
  'dilations': (1, 1, 1, 1),
  'strides': (1, 2, 2, 2),
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'norm_eval': False,
  'style': 'pytorch',
  'contract_dilation': True},
 'decode_head': {'type': 'CustomUPerHead',
  'in_channels': [256, 512, 1024, 2048],
  'in_index': [0, 1, 2, 3],
  'pool_scales': (1, 2, 3, 6),
  'channels': 512,
  'dropout_ratio': 0.1,
  'num_classes': 1,
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'align_corners': False},
 'train_cfg': {},
 'test_cfg': {'mode': 'whole'}}